In [1]:
import os
import sys
from pathlib import Path

# Force correct project root
project_root = Path("C:/Users/tilak/Desktop/TEXT-SUMMARIZER").resolve()
os.chdir(project_root)
print("Current working directory set to:", os.getcwd())

# Add the `src` folder to Python path
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("SRC path added to sys.path:", src_path)


Current working directory set to: C:\Users\tilak\Desktop\Text-Summarizer
SRC path added to sys.path: C:\Users\tilak\Desktop\Text-Summarizer\src


In [2]:
%pwd

'C:\\Users\\tilak\\Desktop\\Text-Summarizer'

In [3]:
%pwd

'C:\\Users\\tilak\\Desktop\\Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    local_data_file: Path
    unzip_dir: Path
    

In [5]:
from textsummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textsummarizer.utils.common import read_yaml, create_directories

print(CONFIG_FILE_PATH.exists(), CONFIG_FILE_PATH)
print(PARAMS_FILE_PATH.exists(), PARAMS_FILE_PATH)


True config\config.yaml
True params.yaml


In [6]:
class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config =DataIngestionConfig(
            root_dir =config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config



In [7]:
import os
import urllib.request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
            self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                      url =self.config.source_url,
                      filename =self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info:\n{headers}")
        else:
             logger.info(f"File already exists at size of :{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
         """ zip_file_path: str
         Extracts the zip file into the specified directory.
         Function will returns None if the extraction is successful, otherwise it will raise an exception.
         """

         unzip_path = self.config.unzip_dir
         os.makedirs(unzip_path, exist_ok =True)
         with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
              zip_ref.extractall(unzip_path)             

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-14 16:29:38,273]:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-12-14 16:29:38,278]:INFO:common: yaml file: params.yaml loaded successfully]
[2025-12-14 16:29:38,280]:INFO:common: created directory at: artifacts]
[2025-12-14 16:29:38,281]:INFO:common: created directory at: artifacts/data_ingestion]
[2025-12-14 16:29:38,282]:INFO:812940721: File already exists at size of :~ 7718 KB]
